In [1]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


C:\Users\anaconda\envs\Pytorch_19\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [3]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [4]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [5]:
print(df_train.shape,df_test.shape)

(19922, 5) (5078, 3)


In [6]:
df_train.head()

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock


In [7]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [8]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [9]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [10]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [11]:
print(df_train.shape,df_test.shape)

(19908, 5) (5078, 3)


### Upsample minority classes

In [12]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [13]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [14]:
df_train.tail()#,df_train.shape

,song_id,filename,filepath,genre_id,genre
19903,9337,009337.ogg,train/009337.ogg,0,Electronic
19904,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19905,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19906,11875,011875.ogg,train/011875.ogg,2,Punk
19907,6579,006579.ogg,train/006579.ogg,1,Rock


In [15]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

#### condense bottom 8 classes into single class

In [16]:
# df_train['genre_id'] = df_train['genre_id'].apply(lambda x: 5 if x>=5 else x).values

In [17]:
# df_train['genre_id'].value_counts()

### Create Folds

In [18]:
df_train['fold'] = 0

In [19]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [20]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [21]:
df_test.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [22]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
    [torchvision.transforms.CenterCrop((64,960))])



In [23]:
import torchaudio

### Make noisy label dataset

In [24]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'efficientnet_b1_CQT_512_64'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


C:\Users\User\AppData\Local\Temp\ipykernel_16772\356762774.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_16772\356762774.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_16772\356762774.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_16772\356762774.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users

In [25]:
f = np.zeros((len(df_test_valid),19))
for fld in range(5):
    for kernels in [
        'efficientnet_b1_CQT_512_64'
                   ]:
        fname = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)
        fname = np.concatenate(fname)
         
        f += fname/5
        
f = pd.DataFrame(f)
f['ID'] = df_test_valid['song_id']
f.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
0,0.102800,0.315599,0.061323,0.101216,0.008844,0.060825,0.013148,0.249553,0.033489,0.001177,0.034693,0.006535,0.000079,0.008373,0.000346,0.000496,0.000267,0.001183,0.000053,7072
1,0.002587,0.000595,0.001668,0.001153,0.001181,0.001294,0.001235,0.001150,0.001242,0.001392,0.001970,0.001808,0.982052,0.000182,0.000154,0.000028,0.000076,0.000111,0.000121,10207
2,0.102242,0.070943,0.038725,0.574358,0.020183,0.003930,0.049830,0.041104,0.061910,0.001389,0.027745,0.001027,0.000170,0.005165,0.000004,0.000035,0.001068,0.000138,0.000033,20008
3,0.589637,0.012913,0.008808,0.020313,0.011665,0.001262,0.271363,0.005349,0.012013,0.011880,0.045817,0.002362,0.002325,0.000877,0.000031,0.002490,0.000037,0.000265,0.000592,10924
4,0.044480,0.027456,0.025329,0.013400,0.843469,0.007058,0.003895,0.004590,0.009626,0.008848,0.002012,0.000051,0.000187,0.000918,0.000512,0.001433,0.004664,0.001720,0.000351,21896


In [26]:
consolNoisy = pd.concat([OOFOut,f],0).reset_index(drop=True)
consolNoisy.head()

C:\Users\User\AppData\Local\Temp\ipykernel_16772\4161917736.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  consolNoisy = pd.concat([OOFOut,f],0).reset_index(drop=True)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
0,0.018749,0.277931,0.090632,0.228058,0.007619,0.050603,1.635911e-03,0.032846,0.045661,0.053363,1.101834e-03,0.001676,0.000324,0.092930,0.010375,7.650562e-04,0.048040,0.037571,0.000119,10150
1,0.000135,0.000369,0.000028,0.000577,0.000004,0.000302,8.308620e-07,0.000011,0.001358,0.982591,3.590461e-07,0.000034,0.000078,0.009454,0.003736,6.186346e-04,0.000045,0.000621,0.000037,11198
2,0.605147,0.016475,0.005267,0.006609,0.200183,0.003854,3.105144e-02,0.007769,0.041302,0.009446,6.345434e-02,0.000268,0.000188,0.000486,0.000013,6.607295e-03,0.000053,0.000142,0.001685,13166
3,0.074297,0.007029,0.012253,0.013157,0.846907,0.000959,3.041148e-03,0.001877,0.009162,0.017448,7.476857e-03,0.000092,0.002987,0.000539,0.000013,3.594669e-04,0.001243,0.000177,0.000983,12275
4,0.073590,0.016229,0.002264,0.814700,0.001749,0.003986,3.473580e-02,0.023601,0.004329,0.001441,1.137127e-02,0.000338,0.000031,0.009079,0.000006,8.699341e-07,0.002296,0.000247,0.000004,24102


In [27]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df,
                 noisyDF = consolNoisy,
                 train = True,
                 valid = False,
                 transform=tfms,

                ):
        
        self.df = df
        self.train = train
        self.valid = valid
        self.transform = transform
        self.noisyDF = noisyDF
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            CQT = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/CQT/512_64/train/{filename}')
        elif self.valid:
            CQT = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/CQT/512_64/train/{filename}')
        else:
            CQT = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/CQT/512_64/test/{filename}')
            
        return CQT


    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
        samples = torch.from_numpy(samples).float()
        
        if self.transform is not None:
            samples = self.transform(samples)
            
        if self.train:
            labels2 = self.noisyDF[self.noisyDF.ID==row.song_id].drop('ID',1).values[0]
            label = torch.tensor(row.genre_id,dtype=torch.long)
            label2 = torch.tensor(labels2).float()
            return samples, (label,label2)
        if self.valid:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:
            return samples

#### Loss

In [28]:
class CombinedLoss:
    "Dice and Focal combined"
    def __init__(self, axis=1, smooth=1., alpha=0.5):
        store_attr()
        self.focal_loss = CrossEntropyLossFlat()
        self.dice_loss =  BCEWithLogitsLossFlat()
        
    def __call__(self, pred, targ):
        if type(targ) is list:
            loss = self.focal_loss(pred, targ[0])*(1-self.alpha) + self.alpha * self.dice_loss(pred, targ[1])
        else:
            loss = self.focal_loss(pred, targ)
        return loss
    
    def decodes(self, x):    return x.argmax(dim=self.axis)
    def activation(self, x): return F.softmax(x, dim=self.axis)

#### FastAI model

In [29]:
import timm
timm.create_model('efficientnet_b2', pretrained=False, in_chans=1)

EfficientNet(
  (conv_stem): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
   

In [30]:
NCLASS = df_train['genre_id'].nunique()
print(NCLASS)

19


In [31]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'efficientnet_b2_CQT_512_64_Noisy'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit =  CombinedLoss()#LabelSmoothingCrossEntropy(0.02) #CrossEntropyLossFlat() #FocalLoss() #CrossEntropyLossFlat() #FocalLoss() #LabelSmoothingCrossEntropy(0.01)
        batch_size = 48
        n_epochs = 15

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms,train=False,valid=True)

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('efficientnet_b2',in_chans=1,pretrained=False)
        model.conv_stem.stride = (1,1)
        model.classifier = nn.Linear(1408,NCLASS)
    
        f1_score = F1Score(average="micro")

        learn = Learner(dls, model,opt_func=Adam,loss_func=crit,metrics=[f1_score],cbs=[SaveModelCallback('f1_score', every_epoch=True,),
                                                                                          CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
        
        learn.fit_one_cycle(n_epochs, 1e-2,wd=1e-3)
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           train=False,
                           valid=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [32]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
if __name__=='__main__':
    run()

*****************************************
Training Fold 0
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.073946,2.410069,0.330236,03:05
1,1.034384,2.442338,0.304621,02:42
2,0.989824,1.985515,0.364139,02:43
3,0.951436,2.008520,0.392014,02:43
4,0.899053,1.806214,0.419136,02:43
5,0.861790,1.633904,0.462079,02:43
6,0.828717,1.614492,0.482672,02:43
7,0.795884,1.614480,0.470116,02:42
8,0.760203,1.545379,0.508790,02:42
9,0.718949,1.530667,0.513812,02:42


*****************************************
Training Fold 1
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.067443,2.092764,0.363636,02:42
1,1.041527,2.544306,0.274234,02:42
2,0.995989,2.084269,0.341286,02:42
3,0.947977,1.833872,0.412356,02:42
4,0.907846,1.969906,0.401557,02:42
5,0.867931,1.784346,0.436966,02:42
6,0.836655,1.641877,0.476143,02:42
7,0.799081,1.570735,0.493973,02:43
8,0.762200,1.531865,0.507534,02:43
9,0.717505,1.549261,0.506027,02:42


*****************************************
Training Fold 2
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.078191,2.114967,0.328729,02:42
1,1.041891,1.915143,0.368910,02:42
2,0.999906,2.067515,0.329483,02:42
3,0.939702,1.802112,0.399548,02:42
4,0.900392,1.838563,0.397037,02:42
5,0.862754,1.658797,0.452536,02:41
6,0.830018,1.573393,0.479407,02:42
7,0.789981,1.552656,0.489453,02:43
8,0.749584,1.504074,0.508790,02:42
9,0.701963,1.518824,0.503516,02:42


*****************************************
Training Fold 3
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.070594,1.994416,0.351922,02:48
1,1.026817,1.959521,0.368752,02:43
2,0.994325,1.900561,0.393369,02:43
3,0.943486,1.980651,0.366491,02:42
4,0.902047,1.626222,0.455413,02:43
5,0.865560,1.660306,0.441849,02:42
6,0.830189,1.645733,0.454408,02:42
7,0.794162,1.564342,0.487064,02:42
8,0.752996,1.599221,0.474504,02:42
9,0.712842,1.623185,0.477769,02:42


*****************************************
Training Fold 4
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.066381,2.398698,0.299673,02:43
1,1.017238,2.257968,0.377543,02:43
2,0.980315,2.126314,0.386586,02:43
3,0.929530,1.848174,0.414218,02:42
4,0.891605,1.772591,0.434313,02:43
5,0.857527,1.690271,0.455916,02:43
6,0.827113,1.626013,0.471992,02:42
7,0.794054,1.610262,0.475509,02:42
8,0.761805,1.559315,0.487817,02:43
9,0.718268,1.592782,0.474755,02:43


In [34]:
df_test_valid.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


In [35]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

### OOF

In [36]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'efficientnet_b2_CQT_512_64_Noisy'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [37]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [38]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [39]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

0.5109001406469761

In [40]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

genre_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
row_0,,,,,,,,,,,,,,,,,,,
0,1915,161,52,185,310,24,419,114,138,108,408,5,0,23,1,35,0,3,5
1,163,1464,818,185,35,154,22,141,253,59,40,8,1,33,44,9,2,8,0
2,29,794,1489,30,15,17,6,20,70,13,0,0,0,4,19,3,2,8,0
3,206,200,74,924,33,69,75,169,86,48,83,40,9,50,1,1,36,5,4
4,186,36,18,43,1275,7,25,8,38,28,16,0,1,2,1,8,5,3,1
5,29,112,24,67,8,718,1,47,85,42,6,17,0,21,23,4,9,6,0
6,204,13,7,56,20,4,580,17,19,11,31,0,0,2,0,2,0,0,0
7,99,71,12,92,8,64,15,385,38,3,94,56,1,42,1,1,0,1,0
8,87,120,45,27,15,61,18,20,124,37,17,3,0,8,4,9,1,1,1


### Mode based sub

In [41]:
kernel_type = 'efficientnet_b2_CQT_512_64_Noisy'

In [42]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [43]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### Mode based sub: check OOF Score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.metrics import f1_score

out = pd.DataFrame()
for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
#                 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
#                 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
#                 'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'seresnet50t_MonoSpec_Hop512_Mels128',
#                 'seresnet50t_MonoSpec_Hop448_Mels160',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

### Mode based sub

In [ ]:
kernel_type = 'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_8bestmodels_CV_5862.csv',index=False)

In [ ]:
kernel_type

In [ ]:
from sklearn.metrics import f1_score

out = pd.DataFrame()
for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
#                 'resnest50d_MonoSpec_Hop448_Mels160',
#                 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
#                 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
#                 'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

In [ ]:
kernel_type = 'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_5bestmodels_CV_5877.csv',index=False)

### Avg probs

In [ ]:
# np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_0_probs.npy',allow_pickle=True)).shape,
df_test_valid.head()



In [ ]:
out = submission.copy().drop('genre_id',1)
f = np.zeros((df_test_valid.shape[0],19))
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        f += (np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)))/25
newDF = pd.DataFrame()
newDF['song_id'] =  df_test_valid['song_id']
newDF['genre_id'] = np.argmax(f,1)

out = pd.merge(out,newDF,on='song_id',how='left')    
# submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
out.head()

In [ ]:
out.loc[submission['song_id']==22612,'genre_id'] = 1
out.loc[submission['song_id']==24013,'genre_id'] = 0
out['genre_id'] = out['genre_id'].astype(int)
out.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_probEns_5bestmodels_NoCVInfo.csv',index=False)

#### Blind sub

In [ ]:
out = submission.copy().drop('genre_id',1)
f = np.zeros((df_test_valid.shape[0],19))
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        f += (np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)))/25
newDF = pd.DataFrame()
newDF['song_id'] =  df_test_valid['song_id']
newDF['genre_id'] = np.argmax(f,1)

out = pd.merge(out,newDF,on='song_id',how='left')    
# submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
out.head()

In [ ]:
out.loc[submission['song_id']==22612,'genre_id'] = 1
out.loc[submission['song_id']==24013,'genre_id'] = 0
out['genre_id'] = out['genre_id'].astype(int)
out.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/BlindSub_NoCVInfo.csv',index=False)

### End ###